In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [2]:
# import logging
# logging.basicConfig(level=logging.NOTSET)

In [ ]:
chat = chat_history()
llm = get_langchain_llm()
tools = [get_word_length]
llm_with_tools = add_tools(llm, tools)
prompt_template = get_prompt_template("You are very powerful assistant, but bad at calculating lengths of words.")
agent = get_agent(llm_with_tools, prompt_template)
agent_executor = get_agent_executor(agent, tools, verbose=True)

input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat})
chat = chat_history(chat, input1, result["output"])
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat})


In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage

def get_langchain_llm(model="gpt-3.5-turbo", temperature=0):
    return ChatOpenAI(model=model, temperature=temperature)

def add_tools(llm, tools):
    llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])
    return llm_with_tools

def get_prompt_template(description):
    MEMORY_KEY = "chat_history"
    return ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are very powerful assistant, but bad at calculating lengths of words.",
            ),
            MessagesPlaceholder(variable_name=MEMORY_KEY),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

def get_agent(llm_with_tools, prompt_template):
    return (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            ),
            "chat_history": lambda x: x["chat_history"],
        }
        | prompt_template
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser()
    )

def get_agent_executor(agent, tools, verbose=True):
    return AgentExecutor(agent=agent, tools=tools, verbose=verbose)

def chat_history(chat=None, usermessage=None, airesponse=None):
    if chat is None:
        return []
    if usermessage is not None and chat is not None:
        chat.extend([HumanMessage(content=usermessage)])
    if airesponse is not None and chat is not None:
        chat.extend([AIMessage(content=airesponse)])
    return chat

class LangchainAgent():

    def __init__(self, description, verbose=True) -> None:
        self.llm = get_langchain_llm()
        self.tools = []
        self.llm_with_tools = add_tools(self.llm, self.tools)
        self.prompt_template = get_prompt_template(description)
        self.agent = get_agent(self.llm_with_tools, self.prompt_template)
        self.agent_executor = get_agent_executor(self.agent, self.tools, verbose=verbose)
        self.chat = []

    def _invoke(self, input):
        # assert that the tool should not be empty
        assert len(self.tools) > 0
        result = self.agent_executor.invoke({"input": input, "chat_history": self.chat})
        self.chat = chat_history(self.chat, input, result["output"])
        return result["output"]
    
    def add_tools(self, tools):
        self.tools.extend(tools)
        self.llm_with_tools = add_tools(self.llm, self.tools)
        self.agent = get_agent(self.llm_with_tools, self.prompt_template)
        self.agent_executor = get_agent_executor(self.agent, self.tools, verbose=True)

    def set_prompt_template(self, description):
        self.prompt_template = get_prompt_template(description)
        self.agent = get_agent(self.llm_with_tools, self.prompt_template)
        self.agent_executor = get_agent_executor(self.agent, self.tools, verbose=True)

In [33]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

In [34]:
la = LangchainAgent("You are very powerful assistant, but bad at calculating lengths of words.")
# add tools
la.add_tools([get_word_length])
la._invoke("how many letters in the word educa?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


'There are 5 letters in the word "educa".'

In [35]:
la._invoke("Is it a real world?")



> Entering new AgentExecutor chain...
I apologize for the confusion. "Educa" is not a recognized English word. It appears to be a misspelling or a word in a different language.

> Finished chain.


'I apologize for the confusion. "Educa" is not a recognized English word. It appears to be a misspelling or a word in a different language.'

In [36]:
la._invoke("What about educat?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educat'}`


6There are 6 letters in the word "educat".

> Finished chain.


'There are 6 letters in the word "educat".'

In [ ]:
from langchain.agents import tool

# define tools with decorator @tool and then add them to the list

tools = []

In [26]:
chat = chat_history()
llm = get_langchain_llm()
tools = [get_word_length]
llm_with_tools = add_tools(llm, tools)
prompt_template = get_prompt_template("You are very powerful assistant, but bad at calculating lengths of words.")
agent = get_agent(llm_with_tools, prompt_template)
agent_executor = get_agent_executor(agent, tools, verbose=True)

input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat})
chat = chat_history(chat, input1, result["output"])
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat})




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}

In [12]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [13]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [14]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}